In [31]:
# if error persist restart kernel

In [2]:
import tensorflow as tf
import os
import tfx
from tfx.components import ImportExampleGen , CsvExampleGen
from tfx.orchestration import metadata
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip



In [3]:
base_dir = os.getcwd()
Tf_record_data_dir = os.path.join(os.pardir, "tfrecord_data")

In [4]:
print(base_dir)
print(Tf_record_data_dir) # one level up(means one directory behind) i.e building-machine-lerarning-pipeline folder

C:\Users\ASUS\building-machine-learning-pipelines\Untitled Folder
..\tfrecord_data


In [5]:
tf_record_location = os.path.join(base_dir, Tf_record_data_dir)
tf_record_location

'C:\\Users\\ASUS\\building-machine-learning-pipelines\\Untitled Folder\\..\\tfrecord_data'

In [6]:
example_gen = ImportExampleGen(input_base=tf_record_location)#ImportExampleGen because we are impoting Tfrecord file format

In [7]:
context = InteractiveContext(pipeline_root='../pipeline_root_learning/tfx_for_tfrecordData')   # ".." means the parent directory
context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [8]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [9]:
from tfx.components import SchemaGen 
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

https://www.coursera.org/learn/machine-learning-data-lifecycle-in-production/ungradedLab/z0oUL/feature-engineering-pipeline/lab?path=%2Fnotebooks%2FC2_W2_Lab_2_Feature_Engineering_Pipeline.ipynb# `lab 2` 

# Tensorflow Transform

The Feature Engineering Component of TensorFlow Extended

### Define a preprocessing function
The preprocessing function is the most important concept of `tf.Transform`. `The preprocessing function is a logical description of a transformation of the dataset`. `The preprocessing function accepts and returns a dictionary of tensors, where a tensor means Tensor or SparseTensor`. There are two kinds of functions used to define the preprocessing function:



The preprocessing function is the heart of TFX data transformation. It's a function that defines how you clean, format, and prepare your raw data before feeding it to your machine learning model.

`There are two main types of operations you can use within this function:`

`TensorFlow Functions`: These functions directly manipulate your data on-the-fly. Imagine them like basic data processing steps (e.g., converting text to lowercase, scaling numbers).

`anlyzers`: These are pre-processing steps that analyze your entire dataset once to generate constant values used in the transformation. Think of them like pre-calculated settings used during processing (e.g., finding the minimum value in a column for normalization).

Please note that TFT will build a graph out of the
preprocess
function and it will run in its own session. It is expected
preprocessing_fn()
that the function returns a dictionary with the transformed features
as values of the Python dictionary.

In [10]:
## https://medium.com/swlh/understanding-ml-in-production-preprocessing-data-at-scale-with-tensorflow-transform-90aefc26e98f

`Transform` will take as input the data from `ExampleGen`, the schema from `SchemaGen`, as well as a `module` containing the preprocessing function.


The pipeline needs to load this as a `module` so you need to use the magic command `%% writefile` to save the file to disk. Let's first define a few `constants` that group the data's attributes according to the `transforms` we will perform later. This file will also be saved locally.

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("../Untitled Folder/data/housing.csv")
data.sample()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
8112,-118.2,33.79,47.0,2549.0,626.0,1388.0,606.0,3.0135,192700.0,NEAR OCEAN


In [13]:
# Set the constants module filename
_housing_constants_module_file = 'housing_constant.py'

In [14]:
# %%writefile {_housing_constants_module_file}

# Features with string data types that will be converted to indices
CATEGORICAL_FEATURE_KEYS = [
    'ocean_proximity'
]

# Numerical features that are marked as continuous
NUMERIC_FEATURE_KEYS = ['longitude','latitude','total_rooms','total_bedrooms','population','households','median_income','median_house_value']

# Feature that can be grouped into buckets
BUCKET_FEATURE_KEYS = ['housing_median_age']

# Number of buckets used by tf.transform for encoding each bucket feature.
FEATURE_BUCKET_COUNT = {'housing_median_age':6}

# Feature that the model will predict
LABEL_KEY = 'ocean_proximity'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Overwriting housing_constant.py


the module that contains `preprocessing_fn()` defines how we will transform the `raw data` into features that your model can train on (i.e. the next step in the pipeline). we will use the tft module functions to make these transformations.

In [15]:
# Set the transform module filename
_housing_transform_module_file = 'housing_transform.py'

In [55]:
# %%writefile {_housing_transform_module_file}


import tensorflow as tf
import tensorflow_transform as tft

import housing_constant

# Unpack the contents of the constants module
_NUMERIC_FEATURE_KEYS = housing_constant.NUMERIC_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = housing_constant.CATEGORICAL_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = housing_constant.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = housing_constant.FEATURE_BUCKET_COUNT
_LABEL_KEY = housing_constant.LABEL_KEY
_transformed_name = housing_constant.transformed_name


# Define the transformations
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}

    # Scale these features to the range [0,1]
    for key in _NUMERIC_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_0_1( # appending transformed features into empy dictionary
            inputs[key])
    
    # Bucketize these features
    for key in _BUCKET_FEATURE_KEYS:
        bucketized_feature = tft.bucketize(inputs[key], _FEATURE_BUCKET_COUNT[key])
        outputs[_transformed_name(key)] = tf.cast(bucketized_feature, tf.float32)

    # Convert strings to indices in a vocabulary
    # for key in _CATEGORICAL_FEATURE_KEYS:
    #     outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

    # Convert the label strings to an index
    outputs[_transformed_name(_LABEL_KEY)] = tft.compute_and_apply_vocabulary(inputs[_LABEL_KEY],top_k = 5)

    return outputs


we can now pass the `training data`, `schema`, and `transform module` to the `Transform component`. we can ignore the warning messages generated by Apache Beam regarding type hints.

# Viewing Tfrecord file

after running example_gen

The output of the components are called artifacts and you can see an example by navigating through .component.outputs > ['examples'] > Channel > ._artifacts > [0] above. It shows information such as where the converted data is stored (.uri) and the splits generated (.split_names).

You can also examine the output artifacts programmatically with the code below

In [56]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0] # 0 is the execution id associated with that dataset. If you restart the kernel of this workspace and re-run up to this cell, you will notice a new folder with a different id name created. This shows that TFX is keeping versions of your data so you can roll back if you want to investigate a particular execution.

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: ../pipeline_root_learning/tfx_for_tfrecordData\ImportExampleGen\examples\1


 the ingested data is stored in the directory shown in the `uri` field. It is also compressed using `gzip` and you can verify by running the cell below.

In [57]:

train_uri = os.path.join(artifact.uri, 'Split-train')

# See the contents of the `train` folder
os.listdir(train_uri)

['data_tfrecord-00000-of-00001.gz']

it may be useful to examine a few examples of the data especially if you're still experimenting. Since the data collection is saved in [TFRecord format](https://www.tensorflow.org/tutorials/load_data/tfrecord), you will need to use methods that work with that data type. You will need to unpack the individual examples from the `TFRecord` file and format it for printing. Let's do that in the following cells:

In [58]:
# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [59]:
from google.protobuf.json_format import MessageToDict

# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

In [60]:
# Get 3 records from the dataset
sample_records = get_records(dataset, 10)

# Print the output
sample_records

[{'features': {'feature': {'total_bedrooms': {'floatList': {'value': [5335.0]}},
    'latitude': {'floatList': {'value': [32.910175]}},
    'households': {'floatList': {'value': [2056.0]}},
    'median_income': {'floatList': {'value': [3.097182]}},
    'population': {'floatList': {'value': [14622.0]}},
    'ocean_proximity': {'bytesList': {'value': ['TkVBUiBPQ0VBTg==']}},
    'total_rooms': {'floatList': {'value': [36985.0]}},
    'median_house_value': {'floatList': {'value': [52418.797]}},
    'housing_median_age': {'floatList': {'value': [8.0]}},
    'longitude': {'floatList': {'value': [-120.38241]}}}}},
 {'features': {'feature': {'median_income': {'floatList': {'value': [7.6961374]}},
    'latitude': {'floatList': {'value': [32.66945]}},
    'longitude': {'floatList': {'value': [-116.77577]}},
    'housing_median_age': {'floatList': {'value': [20.0]}},
    'ocean_proximity': {'bytesList': {'value': ['SVNMQU5E']}},
    'households': {'floatList': {'value': [3809.0]}},
    'total_bed

# Tftransform

In [61]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'], # tfExample
    schema=schema_gen.outputs['schema'],# Schema
    module_file=os.path.abspath(_housing_transform_module_file)) # preprocessing_fn()



In the code above, the `Transform` component in TensorFlow Extended (TFX) is instantiated with the following arguments:

1. `examples`: This argument specifies the output artifacts of the ExampleGen component, which produces examples in TFRecord format. The `examples` output contains the training examples that will be used for the transformation process.

2. `schema`: This argument specifies the output artifacts of the SchemaGen component, which produces a schema that defines the expected structure of the input data. The `schema` output is used to validate the input data during the transformation process.

3. `module_file`: This argument specifies the path to the Python module file that contains the preprocessing logic defined using TensorFlow Transform (TF Transform). This module file contains the `preprocessing_fn` function, which is a callback function used by TF Transform to perform transformations on the input data.

When the `Transform` component is executed as part of a TFX pipeline, `it takes the TFRecord examples from the ExampleGen output`, `validates them against the schema from the SchemaGen output`, and `then applies the transformations defined in the preprocessing_fn function from the specified module file`. The transformed data is then outputted for further processing, such as training machine learning models.

In [62]:
# Run the component
context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

When we execute the `Transform` component, TFX will apply the transformations
defined in our housing_transform.py (module.py in ebook) module file to the loaded input data, `which was converted
to TFRecord data structures during the data ingestion step`. The component will then
output our transformed data, a transform graph, and the required metadata.

Let's examine the output artifacts of `Transform` (i.e. `.component.outputs` from the output cell above). This component produces several outputs:

* `transform_graph` is the graph that can perform the preprocessing operations. This graph will be included during training and serving to ensure consistent transformations of incoming data.
* `transformed_examples` points to the preprocessed training and evaluation data.
* `updated_analyzer_cache` are stored calculations from previous runs.

**Take a peek at the `transform_graph` artifact.  It points to a directory containing three subdirectories.**

In [63]:
# # Get the uri of the transform graph
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri
# # List the subdirectories under the uri
os.listdir(transform_graph_uri)

['metadata', 'transformed_metadata', 'transform_fn']

* The `metadata` subdirectory contains the schema of the original data.
* The `transformed_metadata` subdirectory contains the schema of the preprocessed data. 
* The `transform_fn` subdirectory contains the actual preprocessing graph. 

we can also take a look at the first three transformed examples using the helper function defined earlier.

In [64]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [65]:
# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 10)

# Print the output
sample_records_xf
# pp.pprint(sample_records_xf)

[{'features': {'feature': {'ocean_proximity_xf': {'int64List': {'value': ['2']}},
    'total_bedrooms_xf': {'floatList': {'value': [0.82777345]}},
    'longitude_xf': {'floatList': {'value': [0.3966746]}},
    'housing_median_age_xf': {'floatList': {'value': [0.0]}},
    'latitude_xf': {'floatList': {'value': [0.03843929]}},
    'median_income_xf': {'floatList': {'value': [0.17912042]}},
    'population_xf': {'floatList': {'value': [0.4121976]}},
    'households_xf': {'floatList': {'value': [0.33782893]}},
    'total_rooms_xf': {'floatList': {'value': [0.94274646]}},
    'median_house_value_xf': {'floatList': {'value': [0.0771539]}}}}},
 {'features': {'feature': {'housing_median_age_xf': {'floatList': {'value': [1.0]}},
    'total_bedrooms_xf': {'floatList': {'value': [0.36943367]}},
    'ocean_proximity_xf': {'int64List': {'value': ['4']}},
    'median_house_value_xf': {'floatList': {'value': [0.6794782]}},
    'total_rooms_xf': {'floatList': {'value': [0.6194652]}},
    'longitude_xf

The `transformed data` and the `transform graph` can be consumed during our nextstep, the component `Trainer`

In [68]:
# Assuming you have already defined `transform` and `model` variables
transform_output = transform.outputs['transformed_examples']

# Get the transformed dataset
transformed_dataset = transform_output.get()





import tensorflow_data_validation as tfdv
eval_dir = '../pipeline_root_learning/tfx_for_tfrecordData/Transform/transformed_examples/6/Split-train/transformed_examples-00000-of-00001.gz'
stats = tfdv.generate_statistics_from_tfrecord(data_location=eval_dir)
stats

datasets {
  num_examples: 14239
  features {
    type: FLOAT
    num_stats {
      common_stats {
        num_non_missing: 14239
        min_num_values: 1
        max_num_values: 1
        avg_num_values: 1.0
        num_values_histogram {
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1423.9
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1423.9
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1423.9
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1423.9
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1423.9
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1423.9
          }
          buckets {
            low_val